In [1]:
import operator
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
kf = KFold(n_splits=3, shuffle=True)

In [3]:
le = preprocessing.LabelEncoder()

In [4]:
clf = DecisionTreeClassifier(max_depth=10)

In [5]:
glove_model = KeyedVectors.load_word2vec_format("D:/wiki.de.vec", binary=False)

In [6]:
glove_model.init_sims(replace=True)

In [7]:
file = open("C:/Users/1/Desktop/checked_compounds_final.txt", "r", encoding="utf-8")
lines = file.readlines()

In [8]:
def decapitalize(word):
    if len(word) > 1:
        word = word[0].lower() + word[1:]
    return word

In [9]:
#no suffixes
vectors = []
types = []

for line in lines:
    if 'drucksen' not in line:
        arr = []
        line = line.strip("\n")
        items = line.split("/")
        v1 = glove_model.wv[decapitalize(items[1])].tolist()
        v2 = glove_model.wv[decapitalize(items[4])].tolist()
        items_new = items[1:3] + v1 + items[4:6] + v2 + [items[7]]
        for item in items_new:
            arr.append(item)
        types.append(items[-1])
        vectors.append(arr)

In [11]:
X = np.array(vectors)
y = np.array(types)
kf.get_n_splits(X)

3

In [12]:
all_predicts = []
tests = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_df = pd.DataFrame(X_train)
    y_train_df = pd.DataFrame(y_train)
    X_test_df = pd.DataFrame(X_test)
    y_test_df = pd.DataFrame(y_test)
    
    X_train_df_DT = X_train_df.apply(le.fit_transform)
    y_train_df_DT = y_train_df.apply(le.fit_transform)  
    X_test_df_DT = X_test_df.apply(le.fit_transform)
    y_test_df_DT = y_test_df.apply(le.fit_transform)
    y_test_df_DT = y_test_df_DT[0].values.tolist()
    
    clf = clf.fit(X_train_df_DT, y_train_df_DT)
    predictions = clf.predict(X_test_df_DT)
    all_predicts.extend(predictions)
    tests.extend(y_test_df_DT)
    
print(classification_report(tests, all_predicts))

             precision    recall  f1-score   support

          0       0.41      0.52      0.46      1099
          1       0.42      0.29      0.35      1115
          2       0.19      0.21      0.20       478

avg / total       0.38      0.37      0.37      2692



In [13]:
#with suffixes
vectors = []
types = []

for line in lines:
    if 'drucksen' not in line:
        arr = []
        line = line.strip("\n")
        items = line.split("/")
        v1 = glove_model.wv[decapitalize(items[1])].tolist()
        v2 = glove_model.wv[decapitalize(items[4])].tolist()
        items_new = items[1:4] + v1 + items[4:7] + v2 + [items[7]]
        for item in items_new:
            arr.append(item)
        types.append(items[-1])
        vectors.append(arr)

In [14]:
X = np.array(vectors)
y = np.array(types)
kf.get_n_splits(X)

3

In [15]:
all_predicts = []
tests = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_df = pd.DataFrame(X_train)
    y_train_df = pd.DataFrame(y_train)
    X_test_df = pd.DataFrame(X_test)
    y_test_df = pd.DataFrame(y_test)
    
    X_train_df_DT = X_train_df.apply(le.fit_transform)
    y_train_df_DT = y_train_df.apply(le.fit_transform)  
    X_test_df_DT = X_test_df.apply(le.fit_transform)
    y_test_df_DT = y_test_df.apply(le.fit_transform)
    y_test_df_DT = y_test_df_DT[0].values.tolist()
    
    clf = clf.fit(X_train_df_DT, y_train_df_DT)
    predictions = clf.predict(X_test_df_DT)
    all_predicts.extend(predictions)
    tests.extend(y_test_df_DT)
    
print(classification_report(tests, all_predicts))

             precision    recall  f1-score   support

          0       0.41      0.53      0.46      1099
          1       0.42      0.36      0.38      1115
          2       0.18      0.13      0.15       478

avg / total       0.38      0.39      0.38      2692

